# Vendas de veículos

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   preco            10000 non-null  float64
 1   vendido          10000 non-null  int64  
 2   idade_do_modelo  10000 non-null  int64  
 3   km_por_ano       10000 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 312.6 KB


In [ ]:
dados.sample(10)

,preco,vendido,idade_do_modelo,km_por_ano
6552,58858.00,1,8,18542.81548
3880,19452.22,1,19,22796.30110
5380,91129.30,1,7,20926.24802
8839,32043.53,1,16,14345.65676
2813,31212.02,1,18,19477.84202
3491,30352.21,1,9,29875.78776
932,93855.84,1,18,15184.12290
8276,64528.31,0,19,24975.34746
4868,62666.64,1,9,23731.32764
7125,66813.99,1,19,8228.55542


### Selecionando variaveis

In [ ]:
x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

### Definindo Baseline com dummy

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

SEED = 158020
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y,
                                                        test_size = 0.25,
                                                        stratify = y)

print(f'Treinaremos com {len(treino_x)} elementos e testaremos com {len(teste_x)} elementos')

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)

acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print(f'A acurácia do dummy stratified foi {acuracia:.2f}%')

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 58.00%


## Aplicando algoritimo Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

SEED = 158020
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y,
                                                        test_size = 0.25,
                                                        stratify = y)

print(f'Treinaremos com {len(treino_x)} elementos e testaremos com {len(teste_x)} elementos')

modelo = DecisionTreeClassifier(max_depth = 2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100

print(f'A acurácia da árvore de decisão foi {acuracia:.2f}%')                 

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia da árvore de decisão foi 71.92%


## A validação cruzada

A técnica de separar uma parte dos dados para treino e outra para testes é chamado de Holdout.

No caso em que o número K é exatamente o número de elementos que temos, é chamado de Leave One Out.

### Usando e avaliando com o cross validate

In [ ]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth = 2)

results = cross_validate(modelo, 
                         x, y,
                         cv = 3,  # quantas fatias do conjunto pra fazer a CrossValidation
                         return_train_score = False)

media = results['test_score'].mean()
std = results['test_score'].std()

acuracia_inferior = (media - 2 * std) * 100
acuracia_superior = (media + 2 * std) * 100

print(f'A acurácia com cross validation (3 folds) foi entre {acuracia_inferior:.2f}% e {acuracia_superior:.2f}%')      

A acurácia com cross validation (3 folds) foi entre 74.99% e 76.57%


### Passando 5 como parâmetro no CrossValidate


Segundo o professor, a literatura aponta que ou 5 ou 10 já é suficiente para a validação.

O valor padrão do cross_validate é 5.

In [ ]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth = 2)

results = cross_validate(modelo, 
                         x, y,
                         cv = 5,  # quantas fatias do conjunto pra fazer a CrossValidation
                         return_train_score = False)

media = results['test_score'].mean()
std = results['test_score'].std()

acuracia_inferior = (media - 2 * std) * 100
acuracia_superior = (media + 2 * std) * 100

print(f'A acurácia com cross validation (5 folds) foi entre {acuracia_inferior:.2f}% e {acuracia_superior:.2f}%')      

A acurácia com cross validation (5 folds) foi entre 75.21% e 76.35%


Notem que o cross_validate() não recebe o parâmetro de aleatoriedade, e esse é o padrão.

Da maneira utilizada por nós, ele é determinístico, e nós sabemos em quantos pedaços ele quebrará os dados, porque somos nós quem decidimos essa quantidade.

Portanto, a realidade é que a aleatoriedade (random) só é aplicada em DecisionTreeClassifier(). E, por padrão, ele só a usará em um caso específico e raro. Por estas razões, mesmo que mudemos constantemente o valor de SEED, obteremos basicamente sempre os mesmos números.

## Kfold com aleatorização

Antes de continuar, uma breve refatoração para exibir os resultados através de uma função.

In [ ]:
def imprime_resultado(results):
  media = results['test_score'].mean()
  std = results['test_score'].std()

  acuracia_inferior = (media - 2 * std) * 100
  acuracia_superior = (media + 2 * std) * 100

  print(f'Acurácia média é de {media * 100:.2f}%')
  print(f'O intervalo da acurácia com cross validation foi entre {acuracia_inferior:.2f}% e {acuracia_superior:.2f}%')      

Por mais que recebamos todos os dados em uma sequência, não queremos quebrar eles em 5 pedaços e, a partir disso, fazer o cross_validate. O ideal seria embaralhar estes dados e, então executar a validação cruzada. O algorítimo pode fazer isso de maneira mais inteligente, sem embaralhar e copiar todo o espaço de memória e todos os objetos que estiverem dentro, e assim por diante. Mas o importante é a ideia de embaralhar esses dados de alguma maneira, e o cross_validate não faz isso de verdade.

O cv aceita como parâmetro, tanto números, quanto geradores de validação cruzada. E existem diversos geradores de validação cruzada. O que usaremos é o KFold, que "corta" nossos dados em K pedaços.

É ele que estamos usando por baixo dos panos. Então, por enquanto, nosso cross_validate é um KFold de n_splits, no caso, igual a 3 ou 5 ou 10, como no exemplo abaixo:

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)  #aqui

modelo = DecisionTreeClassifier(max_depth = 2)

results = cross_validate(modelo, 
                         x, y,
                         cv = cv,  #aqui
                         return_train_score = False)

imprime_resultado(results)

Acurácia média é de 75.78%
O intervalo da acurácia com cross validation foi entre 74.37% e 77.19%


Ainda não conseguimos embaralhar nossos dados, porém, com o objeto KFold criado, será possível fazermos um suffle, pois ele tem um randon_state para ser setado.

Vamos copiar e colar o código e adicionar o shuffle como parâmetro de KFold(), juntamente com n_splits para vermos quais resultado atingiremos com e sem o shuffle.

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10,
           shuffle = True)  #aqui

modelo = DecisionTreeClassifier(max_depth = 2)

results = cross_validate(modelo, 
                         x, y,
                         cv = cv,
                         return_train_score = False)

imprime_resultado(results)

Acurácia média é de 75.76%
O intervalo da acurácia com cross validation foi entre 73.26% e 78.26%


É importante ressaltar que se rodarmos diversas vezes, Accuracy se mantém, pois continua sendo determinístico.

Mas ainda assim, é diferente dos valores apresentados sem o shuffle.

No entanto, repare que a média obtida por meio dos dois algorítimos é bastante próxima, mas o intervalo é diferente.

Na prática, se os dados não possuem alguma estrutura interna, como uma sequência por datas, por exemplo, primeiro gera-se um sequência aleatória desses dados e, em seguida, quebra-se em 10 pedaços ou faz-se ambos ao mesmo tempo. Esta é a forma tradicional de execução com shuffle e n_splits.

Assim, podemos usar não somente o KFold, mas, em seu Model Selection, encontraremos o KFold e diversos outros "quebradores" de grupos, que são as Splitter classes. Estudaremos alguns destes outros adiante.

## Estratificação com validação cruzada

O KFold divide nossos dados em k dobras sem prestar atenção no balanceamento dos dados, e pode cair na condição de seus dados serem treinados com todos os rótulos com valor 1 e quando for usado o conjunto de dados para teste, os dados contenham algum 0 lá, por exemplo.

O StratifiedKFold é um pouco melhor, esse algoritmo faz a mesma coisa que o KFold mas com uma grande melhoria: Obedece ao balanceamento dos nossos rótulos.

### Simulação de dados não balanceados

Para causar um desbalanço, vamos ordenar os rótulos para todos 0 depois 1 e definir um novo DF e variáveis que deram o 'azar' de serem desbalanceadas.

In [ ]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


Agora, vamos rodar o KFold sem o shuffle, e com esses dados de azar.

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth = 2)

results = cross_validate(modelo, 
                         x_azar, y_azar,
                         cv = cv,
                         return_train_score = False)

imprime_resultado(results)

Acurácia média é de 57.84%
O intervalo da acurácia com cross validation foi entre 34.29% e 81.39%


A acurácia é baixíssima e o intervalo gigantesco!

Vamos tentar com o shuffle agora:

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10,
           shuffle = True)

modelo = DecisionTreeClassifier(max_depth = 2)

results = cross_validate(modelo, 
                         x_azar, y_azar,
                         cv = cv,
                         return_train_score = False)

imprime_resultado(results)

Acurácia média é de 75.78%
O intervalo da acurácia com cross validation foi entre 72.30% e 79.26%


Com estes resultados podemos notar que o shuffle nos é suficientemente bom nesta situação em que temos um balanço para recuperar.

Mas e se quiséssemos fazer a estratificação?

## Usando o StratifiedKFold

Além do KFold, temos outros geradores de grupo de separação, dentre eles o StratifiedKFold, que recebe um número de n_splits, shuffle e depois separa, mantendo a proporção entre as classes.

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10,  #aqui
                     shuffle = True)

modelo = DecisionTreeClassifier(max_depth = 2)

results = cross_validate(modelo, 
                         x_azar, y_azar,
                         cv = cv,
                         return_train_score = False)

imprime_resultado(results)

Acurácia média é de 75.78%
O intervalo da acurácia com cross validation foi entre 73.55% e 78.01%


Embora o resultado seja mais fechado, é o mais recomendado. Principalmente quando há um desbalanço entre duas ou mais classes, é interessante utilizar o StratifiedKFold.

Reparem como ficariam os resultados se o deixassemos no aleatório. Inclusive, mesmo sem o shuffle, ele recupera bem a capacidade de prever o quão bom seria o treinamento. Vamos deletar shuffle:

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth = 2)

results = cross_validate(modelo, 
                         x_azar, y_azar,
                         cv = cv,
                         return_train_score = False)

imprime_resultado(results)

Acurácia média é de 75.78%
O intervalo da acurácia com cross validation foi entre 73.83% e 77.73%


Feita essa demonstração, podemos voltar o shuffle ao código, mas será o StrafiedKFold que mais usaremos em casos de desbalanço entre as classes.

## Gerando dados aleatórios

A seguir, criaremos uma coluna 'modelo' (do veiculo) para fins didáticos.

In [ ]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size = 10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


Agora que temos os dados criados, como modelo do carro, precisamos testar nosso estimador.

## Validação cruzada usando grupos com GroupKFold

O que estamos precisamos é: de capacidade para separar treino e teste na validação cruzada, de acordo com os grupos.

In [ ]:
from sklearn.model_selection import GroupKFold

O GroupKFold irá agrupar por grupo, mas é preciso que apontemos quais são estes grupos.

Na documentação do cross_validate, um dos diversos parâmetros é groups, referente aos dados da coluna. Então, anotamos dados.modelo como um parâmetros groups de cross_validate().

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth = 2)

results = cross_validate(modelo, 
                         x, y,
                         cv = cv,
                         groups = dados.modelo,  #aqui
                         return_train_score = False)

imprime_resultado(results)

Acurácia média é de 75.80%
O intervalo da acurácia com cross validation foi entre 72.00% e 79.60%


O valor da média obtida é próximo ao de StratifiedKFold, porque os dados de um modelo de carro estão funcionando de maneria muito parecida, mas isso não é regra. Por exemplo, imagine que passe a existir um carro voador que faz viagens espaciais. A maneira de precificar este carro será totalmente diferente do que este estimador é capaz de classificar.

No caso que estamos trabalhanso, de modo geral, os carros se comportam indiferentemente do modelo. Então, nossa classificação foi próxima, levando em consideração ou não o modelo do carro.

Quando é dito "levando em consideração" não é para estimar, mas sim para treinar e testar.

Reparem que x o x_azar, entre outros, continuam sem usar a coluna modelo. Para classificar só usamos preco, idade_do_modelo e km por ano.

##  A importância do pipeline no crossvalidate

### Para lembrar: padronização e normalização

As duas técnicas tem o mesmo objetivo: transformar todas as variáveis na mesma ordem de grandeza. E a diferença básica é que padronizar as variáveis irá resultar em uma média igual a 0 e um desvio padrão igual a 1. Já normalizar tem como objetivo colocar as variáveis dentro do intervalo de 0 e 1, caso tenha resultado negativo -1 e 1.

[Fonte: Artigo no medium](https://medium.com/data-hackers/normalizar-ou-padronizar-as-vari%C3%A1veis-3b619876ccc9)

Anteriormente, percebemos que a árvore de decisão não precisa primeiramente padronizar ou normalizar, as nossas features. Alguns algorítimos precisam, como o SVC (ou SVM).

Se pensarmos, será que os dados escalados foram utilizados? Ao analisarmos, percebeeremos que não, porque os dados escalados estão em treino_x_escalado e teste_x_escalado e, no caso, em cross_validate() foi utilizado o x_azar, que não está escalado e faz parte dos dados crus.

Para escalar e inseri-los em cross_validate(), vamos reescalar todo o x_azar e utilizá-lo de uma só vez. Antes, vamos pensar mais um pouquinho. Para escalar o x_azar, teremos que criar um StandardScaler, dar um fit() e um transform() nele.

Entretanto, no pré processamento de dados, quando escalamos as features, fit() é aplicado somente nos dados de treino, e aqui o fit() acontece em treino e teste.

Até porque faremos dez separações baseadas nos grupos, para treinar e testar. Ou seja, não faremos o fit() somente uma vez. Devemos fazer esse fit() diversas vezes. Na verdade, deveríamos intercalar, fazendo uma vez esse fit, uma vez a validação, uma vez o fit, outra vez a validação, com outro conjunto de dados.

Toda vez que rodamos o GroupKFold, temos que fazer o fit() com uma parte de treino e rodar a validação com o teste. Se tivermos outro treino, rodaremos fit() para esse outro treino, e o teste com o resto. Assim, teremos um processo de duas fases:

* Primeiro, o pré processamento que escala;
* depois, a validação.

Esse processo deve ser rodado várias vezes, de acordo com os nossos grupos e com nossos com nossos splits. Então, precisamos pensar em outra forma, além de fazer o fit() somente uma vez e, depois, rodar o GroupKFold. Provavelmente, a estimativa obtida por esse processo será muito otimista.

## Pipeline

Sendo assim, no Sklearn, criaremos um processo, uma sequência de passos, chamada Pipeline, que está em um módulo separado. Começaremos importando-o com:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline  #aqui

O Pipeline() será como um processo de dois passos

* o primeiro é o scaler;
* depois, vem o modelo.

Assim, scaler será igual ao StandarScaler(), e o modelo igual ao SVC().

In [ ]:
scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

Reparem que não foi feito o fit(). Apenas criamos o scaler e o modelo. Podemos passá-los dentro de um array ([]), com os nomes transformação e estimador, respectivamente.

Desse modo, se imprimirmos ou analisarmos por dentro, encontraremos: primeiro uma transformação, que é o Standrscaler(), e depois um estimador que é um SVC().

O legal do Pipeline é que ele funciona como se fosse um estimador, então tem o fit() e o predict, que funciona da mesma forma. Então, podemos passar ele para o processo de cross_validate.

Na sequência, definimos GroupKFold() e aplicamos cross_validate() para o pipeline inteiro, utilizando o exemplo de x.

In [ ]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()

modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)

results = cross_validate(pipeline,  #aqui
                         x, y,
                         cv = cv,
                         groups = dados.modelo,
                         return_train_score = False)

imprime_resultado(results)

Acurácia média é de 76.55%
O intervalo da acurácia com cross validation foi entre 73.22% e 79.88%


Agora sim, para cada um dos processos de fold, serão rodados tanto a transformacao, quanto o estimador.

Depois, outro conjunto de dados será separado em dois, tanto a transformacao, quanto o estimador.

Reparem que o valor anterior e este não são tão distantes, mas é preciso entender que a forma sem escala estava errada, além de rodarmos o scaler somente uma vez, para depois rodar os KFolds.

Não é questão de estar melhor ou pior, pois estamos falando de um processo, que deve ser rodado para cada uma das fases de treino. Portanto, se tivermos 10 fases de treino, teremos que rodar o scaler para cada uma do conjunto de treino. E é isso que o Pipeline faz para nós, de forma muito mais prática. Basta inserir todas as fases nele e rodar. Claro, pode demorar um pouco mais, mas é a opção que faz mais sentido.

Para rodar algorítimos com cross_validate, que exigem pré processamento, vocês talvez queiram fazer o treino desse pré processamento somente na parte de treino. Para isso será necessário incluí-lo em um Pipeline.

## Treinando o modelo final

Agora que temos diversas abordagens de validação você vai escolher a que acredita ser ideal para as características dos seus dados, do seu dataset, de acordo com o que deseja predizer.

A média e o intervalo providos pela validação cruzada diz o quão bem você espera que o modelo se comporte com dados previamente desconhecidos mas... se você usou cross validation com 10 folds, qual dos 10 modelos treinados você vai usar agora no mundo real?

A ideia é que a validação cruzada num conjunto de dados somente te diz o que você pode esperar. Ela não treina o seu modelo final.

Agora você tem o algoritmo do estimador (por exemplo um DecisionTreeClassifier) e pretende usá-lo no mundo real:

In [ ]:
modelo = DecisionTreeClassifier(max_depth = 2)

modelo.fit(x, y)  # ajusta TODOS os dados para treinar o modelo

predizer_valores_x = modelo.predict(x)  # pede ao modelo para fazer as pedições para x

Acima passamos o dataframe de x como exemplo para predizer os resultados. No mundo real seria um novo dataframe inteiro, que segue os padrões das colunas. Ou entao os valores singulares (alinhado dentro de um array), como se fosse apenas 1 linha.

In [ ]:
predizer_valores_x

array([1, 1, 0, ..., 0, 1, 0])

Aqui ele prediz todas as linhas de x com os resultados 1 ou 0 (vendido ou não).

Se fosse os dados de apenas uma linha, ele retornaria um array com um único valor (1 ou 0).